In [ ]:
from random import shuffle
import numpy as np
import fasttext
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
def train_test_split(dataframe, labels, method, perc_training, num_trainingsamples):
#     Creates a radomly-sampled train/test split for the provided data (dataframe and labels). Two methods are possible:
#     method = 0: by specified percentage. The size of the training data will be len(data)*perc_training
#     method = 1: by number of cases. The size of the training data will be num_trainingsamples
    
    if(method==0):
        num_trainingsamples = int(round(len(dataframe)*perc_training))
        
    msk = [False]*(len(dataframe)-num_trainingsamples) + [True]*num_trainingsamples
    shuffle(msk)
    msk = np.asarray(msk)
    
    data_train = dataframe[msk]
    data_test = dataframe[~msk]
    labels_train = labels[msk]
    labels_test = labels[~msk]
    return data_train, data_test, labels_train, labels_test, msk

In [ ]:
def enrich(df):
#     Takes the dataframe as loaded from the .csv and enriches it with some features
       
    data = df.copy()
    
    #Adding some extra features
    data['distinct_ratio'] = (data['NUM_DISTINCT'] / (data['NUM_ROWS'] - data['NUM_NULLS'])).replace(np.inf, 0)
    data['perc_null'] = (data['NUM_NULLS'] / data['NUM_ROWS']).replace(np.inf, 0)
    data['is_id'] = np.where(data['COLUMN_NAME'].str.endswith('ID'), 1, 0)
    
    #Transforming the data type into 4 booleans (might have to be changed for data sources other than ORACLE)
    data['is_number'] = np.where(data['DATA_TYPE']=='NUMBER', 1, 0)
    data['is_character'] = np.where(data['DATA_TYPE'].isin(['VARCHAR2','BLOB','CLOB']), 1, 0)
    data['is_date'] = np.where(data['DATA_TYPE'].isin(['DATE','TIMESTAMP(6)']), 1, 0)
    data['is_large_blob'] = np.where(data['DATA_LENGTH']>=1000, 1, 0)
    
    #Drop columns that will not be used in the model
    data = data.drop(['DATA_TYPE'], axis=1)
    data = data.drop(['NUM_DISTINCT'], axis=1)    
    data = data.drop(['NUM_NULLS'], axis=1)
    data = data.drop(['DATA_LENGTH'], axis=1)
    
    #Fill the cells that contain NaN
    data=data.fillna(0)

    return data

Functions related to training and using the text classifier

In [ ]:
def prepare_to_embed(labels,pre_embedding_file):
#     Takes the column names and labels and puts them in the format needed by FastText. They are temporarily saved in a .csv
    df = labels[['PERSONAL_DATA','COLUMN_NAME']].copy()
    df['COLUMN_NAME'] = df['COLUMN_NAME'].str.replace('_', '', regex=True)
    df.insert(loc=0, column='aux', value = '__label__')
    df.insert(loc=2, column='aux2', value = ' ')
    df['final'] = df['aux'] + df['PERSONAL_DATA'].astype(str) + df['aux2'] + df['COLUMN_NAME']
    df['final'].to_csv(pre_embedding_file, index = False, header = False)

In [ ]:
def train_embedding_model(labels,column_to_embed):
#     Trains the text classifier based on the labels and text column provided
    prepare_to_embed(labels,'pre_embedding.csv')
    model = fasttext.train_supervised(input="pre_embedding.csv", epoch = 10, minn=1, maxn=10, lr = 1)
    os.remove('pre_embedding.csv')
    return model

In [ ]:
def predict_label(word,model):
#     Outputs the class prediction of the text classifier
    if(model.predict(word)[0][0] == '__label__0'):
        return 0
    if(model.predict(word)[0][0] == '__label__1'):
        return 1

In [ ]:
def add_embedding_to_df(data,model,column_to_embed):
#     Takes a string column from a dataframe, uses the given text classifier model to create a class prediction, and
#     adds a column to the dataframe with said prediction
    data['embedding_pred'] = np.vectorize(predict_label)(data[column_to_embed],model)
    return data



Functions related to training the main prediction model (by default, a Random Forest Classifier):

In [ ]:
def train_model(training_data, labels, model_type, estimators):
#     Trains a machine learning model with the given data and labels
#     The best results (featured in the paper) were obtained with a random forest (model_type = 1, default)
    if model_type not in range(1,8):
        model_type = 1 #by default, random forest
    
    if model_type == 1: #Random Forest
        ml_model = RandomForestClassifier(n_estimators = estimators)
    elif model_type == 2: #Logistic Regression
        ml_model = LogisticRegression(solver='liblinear')
    elif model_type == 3: #Gaussian Naive Bayes
        ml_model = GaussianNB()
    elif model_type == 4: #Ada Boost
        ml_model = AdaBoostClassifier(n_estimators=estimators)
    elif model_type == 5: #Gradient Boosting
        ml_model = GradientBoostingClassifier(n_estimators=estimators, learning_rate=1.0, max_depth=3, random_state=0)
    elif model_type == 6: #KNN classifier
        ml_model = KNeighborsClassifier(n_neighbors=9)
        
    ml_model.fit(training_data, labels.values.ravel())
    return ml_model

In [ ]:
def calc_importances_df(model,data):
    #Returns a dataframe with the feature importances
    return pd.DataFrame(model.feature_importances_,index = data.columns, columns=['importance']).sort_values('importance',ascending=False)

In [ ]:
def predict(model, data_test):
    #Returns a dataframe with the predictions made by the model on the data_test
    return pd.DataFrame(model.predict_proba(data_test)[:,1])

In [ ]:
def evaluate_prediction(test_columns,predictions,labels,threshold,results_file):
#     Compares the predictions iwth the labels
#     Returns the precision and recall and f1 based on the provided threshold, as well as the ROC AUC score
#     Saves the results to the results_file, where they can be analyzed
    to_eval = labels.reset_index().join(predictions.reset_index().drop(['index'], axis=1)).drop(['index'], axis=1)
    to_eval.columns = ['labels','Prediction']
    to_eval['Prediction (binary)'] = np.where(to_eval['Prediction']>=threshold, 1, 0)
    to_write = test_columns.reset_index().join(to_eval.reset_index().drop(['index'], axis=1))
    to_write.to_csv(results_file, index=False)
    
    if len(to_eval[to_eval['Prediction (binary)'] == 1]) > 0:
        precision = len(to_eval[(to_eval['labels'] == 1) & (to_eval['Prediction (binary)'] == 1)])/len(to_eval[to_eval['Prediction (binary)'] == 1])
    else:
        precision = 0
    
    if len(to_eval[to_eval['labels'] == 1]) > 0: 
        recall = len(to_eval[(to_eval['labels'] == 1) & (to_eval['Prediction (binary)'] == 1)])/len(to_eval[to_eval['labels'] == 1])
    else:
        recall = 0
    
    if ((precision >0) and (recall> 0)):
        f1 = 2*(precision*recall)/(precision+recall)
    else:
        f1 = 0
        
    auc = roc_auc_score(to_eval['labels'], to_eval['Prediction (binary)'])
    
    return precision,recall,f1,auc

Functions related to the "Advanced Options" section:

In [ ]:
def full_run(specify_test_data, split_method, perc_training, num_trainingsamples, columns_to_use, model_type, results_file, num_tests):
    data = pd.read_csv(metadata_file)
    labels = pd.read_csv(labels_file)

    #OPTIONAL, but used for the results in the paper:
    data = data[data['NUM_DISTINCT'] > 0] #We'll only analyze columns for which there were some entries
    labels = labels[labels.index.isin(data.index)] #Make sure the data and labels match

    precision_list = list()
    recall_list = list()
    f1_list = list()
    auc_list = list()
    
    for i in range(0,num_tests):
        if(specify_test_data == 0):
            data_train, data_test, labels_train, labels_test, msk = train_test_split(data, labels, split_method, perc_training, num_trainingsamples)
        elif(specify_test_data == 1):
            data_train = data
            labels_train = labels
            data_test = pd.read_csv(metadata_file_test)
            labels_test = pd.read_csv(labels_file_test)
            data_test = data_test[data_test['NUM_DISTINCT'] > 0]
            labels_test = labels_test[labels_test.index.isin(data_test.index)]

        data_train = enrich(data_train)
        data_test = enrich(data_test)

        if('embedding_pred' in columns_to_use):
            embed_model = train_embedding_model(labels_train,'COLUMN_NAME')
            data_train = add_embedding_to_df(data_train,embed_model,'COLUMN_NAME')
            data_test = add_embedding_to_df(data_test,embed_model,'COLUMN_NAME')

        test_columns = labels_test[['TABLE_NAME','COLUMN_NAME']]
        data_train = data_train[columns_to_use]
        data_test = data_test[columns_to_use]
        labels_train = labels_train.drop(['TABLE_NAME','COLUMN_NAME'], axis=1)
        labels_test = labels_test.drop(['TABLE_NAME','COLUMN_NAME'], axis=1)

        ml_model = train_model(data_train, labels_train, model_type, 100)
        predictions = predict(ml_model, data_test)

        precision,recall,f1,auc = evaluate_prediction(test_columns,predictions,labels_test,0.5,results_file)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)
        
    mean_precision = sum(precision_list)/len(precision_list)
    mean_recall = sum(recall_list)/len(recall_list)
    mean_f1 = sum(f1_list)/len(f1_list)
    mean_auc = sum(auc_list)/len(auc_list)
    
    return mean_precision, mean_recall, mean_f1, mean_auc